# Fine-tuning de Llama 2 pour l'analyse de sentiments

**Pour ce tutoriel pratique sur le fine-tuning de Llama 2, nous allons traiter d'une analyse de sentiments sur des informations financières et économiques.**   

#### Problématique
L'analyse de sentiments sur des informations financières et économiques est cruciale pour les entreprises. Elle fournit des informations précieuses sur les **tendances du marché, la confiance des investisseurs et le comportement des consommateurs.** De plus, elle permet d'identifier les risques potentiels pour la réputation et aide à évaluer le sentiment des parties prenantes, des investisseurs et du grand public, facilitant ainsi la prise de décisions d'investissement éclairées.

Avant d'aborder les aspects techniques du finetuning d'un LLM comme Llama 2, nous devons trouver l'ensemble de données adéquat pour démontrer les possibilités et capacités du finetuning.

#### Dataset
L'ensemble de données **FinancialPhraseBank** est une collection complète qui capture les sentiments des titres de l'actualité financière du point de vue d'un investisseur. Comprenant deux colonnes clés, à savoir **"Sentiment"** et **"Titre de l'actualité"**, l'ensemble de données classe efficacement les sentiments comme étant **soit négatifs, soit neutres, soit positifs**.  

[Consultez la carte du Dataset ici](https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news)

## Bibliothèques & Installations 🗂


Les bibliothèques spécifiques nécessaires sont entre autres:

* **accelerate:** bibliothèque d'entraînement distribué pour PyTorch par HuggingFace. Elle vous permet d'entraîner vos modèles sur plusieurs GPU ou CPU en parallèle (configurations distribuées), ce qui peut accélérer considérablement l'entraînement en présence de plusieurs GPU (nous ne l'utiliserons pas dans notre exemple).

* **peft:** bibliothèque Python par HuggingFace pour l'adaptation efficace des LLMs sans ajuster tous les paramètres du modèle. Les méthodes PEFT n'ajustent qu'un petit nombre de paramètres (supplémentaires) du modèle, réduisant ainsi considérablement les coûts computationnels et de stockage.   
[**Détails PEFT:** Voir ressources de la Séance 7](https://github.com/ANYANTUDRE/Stage-IA-Selever-GO-AI-Corp/blob/main/02.%20Supports%20de%20Cours%20-%20Formations/07.%20S%C3%A9ance%207%20-%20M%C3%A9triques%20%26%20%20PEFTs%20-%20LoRA%20%26%20QLoRA/03.%20Parameter%20efficient%20Fine-tuning%20(PEFT).pdf)

* **bitsandbytes:** wrapper léger autour des fonctions personnalisées CUDA, en particulier les optimisateurs 8 bits, la multiplication matricielle, et les fonctions de quantification. Il permet d'exécuter des modèles stockés en précision 4 bits : bien que bitsandbytes stocke les poids en 4 bits, le calcul se fait toujours en 16 ou 32 bits et ici, toute combinaison peut être choisie (float16, bfloat16, float32, etc.).    
[**Détails Quantization:** Voir ressources de la Séance 5](https://github.com/ANYANTUDRE/Stage-IA-Selever-GO-AI-Corp/blob/main/02.%20Supports%20de%20Cours%20-%20Formations/05.%20S%C3%A9ance%205%20-%20Quantization%20%26%20Rappels/02.%20Technique%20de%20Quantization.pdf)

* **transformers:** bibliothèque pour le traitement du langage naturel (NLP). Elle fournit un certain nombre de modèles pré-entraînés pour des tâches NLP telles que la classification de texte, les réponses aux questions, et la traduction automatique.  
[**Détails Transformers:** Voir ressources de la Séance 4](https://github.com/ANYANTUDRE/Stage-IA-Selever-GO-AI-Corp/tree/main/02.%20Supports%20de%20Cours%20-%20Formations/04.%20S%C3%A9ance%204%20-%20Utilisation%20des%20Transformers%20de%20Hugging%20Face)

* **trl:** bibliothèque complète par HuggingFace fournissant un ensemble d'outils pour entraîner des modèles de langage de type transformer avec l'apprentissage par renforcement, depuis l'étape de fine-tuning supervisé (SFT), l'étape de modélisation de la récompense (RM) jusqu'à l'étape d'optimisation de la politique proximale (PPO).   
[**Documentation officielle**](https://huggingface.co/docs/trl/sft_trainer)

In [1]:
!pip install -q -U "torch==2.1.2" tensorboard

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.12 requires torch<2.1,>=1.7, but you have torch 2.1.2 which is incompatible.
tensorflow 2.12.0 requires tensorboard<2.13,>=2.12, but you have tensorboard 2.17.0 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.1.2 which is incompatible.


In [2]:
!pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"

In [3]:
!pip install -q -U git+https://github.com/huggingface/trl@a3c5b7178ac4f65569975efadc97db2f3749c65e
!pip install -q -U git+https://github.com/huggingface/peft@4a1559582281fc3c9283892caea8ccef1d6f5a4f

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
chex 0.1.82 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cudf 23.8.0 requires dask==2023.7.1, but you have dask 2

La cellule suivante importe le module os et définit deux variables d'environnement :
* **CUDA_VISIBLE_DEVICES** : variable d'environnement indiquant à PyTorch quels GPU utiliser. Dans ce cas, le code définit la variable d'environnement à 0, ce qui signifie que PyTorch utilisera le premier GPU.
* **TOKENIZERS_PARALLELISM** : variable d'environnement indiquant à la bibliothèque Hugging Face Transformers s'il faut paralléliser le processus de tokenisation. Dans ce cas, le code définit la variable d'environnement à false, ce qui signifie que le processus de tokenisation ne sera pas parallélisé.

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
import warnings
warnings.filterwarnings("ignore") ### filtrer et ignorer les avertissements inutiles

In [6]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer,setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [7]:
print(f"PyTorch version --> {torch.__version__}")

PyTorch version --> 2.1.2+cu121


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device --> {device}")

Device --> cuda:0


## Préparation des données 🛢 & utilitaires 

Le code dans la cellule suivante effectue les étapes suivantes :

1. Lit le jeu de données d'entrée à partir du fichier all-data.csv, qui est un fichier CSV avec deux colonnes : **sentiment et texte.**
2. Divise le jeu de données en ensembles d'entraînement et de test, avec 300 échantillons dans chaque ensemble. La division est stratifiée par sentiment, de sorte que chaque ensemble contient un échantillon représentatif de sentiments positifs, neutres et négatifs.
3. Mélange les données d'entraînement dans un ordre reproductible (random_state=2024).
4. Transforme les textes contenus dans les données d'entraînement et de test en prompts à utiliser par Llama : les prompts d'entraînement contiennent la réponse attendue avec laquelle nous voulons affiner le modèle.
5. Les exemples résiduels qui ne sont pas dans l'entraînement ou le test, à des fins de reporting pendant l'entraînement (mais qui ne seront pas utilisés pour l'arrêt anticipé), sont traités comme des données d'évaluation, qui sont échantillonnées avec répétition afin d'avoir un échantillon de 50/50/50 (les instances négatives étant très peu nombreuses, elles doivent donc être répétées).
6. Les données d'entraînement et d'évaluation sont encapsulées par la classe de Hugging Face (https://huggingface.co/docs/datasets/index).

**Cela prépare en une seule cellule les jeux de données train_data, eval_data et test_data à utiliser dans notre fine-tuning.**

In [9]:
filename = "../input/sentiment-analysis-for-financial-news/all-data.csv"

df = pd.read_csv(filename, 
                 names=["sentiment", "text"],
                 encoding="utf-8", encoding_errors="replace")

X_train = list()
X_test = list()
for sentiment in ["positive", "neutral", "negative"]:
    train, test  = train_test_split(df[df.sentiment==sentiment], 
                                    train_size=300,
                                    test_size=300, 
                                    random_state=2024)
    X_train.append(train)
    X_test.append(test)

X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)

eval_idx = [idx for idx in df.index if idx not in list(X_train.index) + list(X_test.index)]
X_eval = df[df.index.isin(eval_idx)]
X_eval = (X_eval
          .groupby('sentiment', group_keys=False)
          .apply(lambda x: x.sample(n=50, random_state=10, replace=True)))
X_train = X_train.reset_index(drop=True)

def generate_prompt(data_point):
    return f"""
            Analyse le sentiment du titre de l'actualité inclus entre crochets, 
            détermine s'il est positif, neutre ou négatif, et retourner la réponse 
            sous forme d'étiquette de sentiment correspondante "positive", "neutral" ou "negative".

            [{data_point["text"]}] = {data_point["sentiment"]}
            """.strip()

def generate_test_prompt(data_point):
    return f"""
            Analyse le sentiment du titre de l'actualité inclus entre crochets, 
            détermine s'il est positif, neutre ou négatif, et retourner la réponse 
            sous forme d'étiquette de sentiment correspondante "positive", "neutral" ou "negative".

            [{data_point["text"]}] = """.strip()

X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1), 
                       columns=["text"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1), 
                      columns=["text"])

y_true = X_test.sentiment
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)

Ensuite, nous créons une fonction pour évaluer les résultats de notre modèle de sentiment affiné. La fonction effectue les étapes suivantes :

1. Associe les étiquettes de sentiment à une représentation numérique, où **2 représente positif, 1 représente neutre, et 0 représente négatif.**
2. Calcule la précision du modèle sur les données de test.
3. Génère un rapport de précision pour chaque étiquette de sentiment.
4. Génère un rapport de classification pour le modèle.
5. Génère une matrice de confusion pour le modèle.

In [10]:
def evaluate(y_true, y_pred):
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 2, 'neutral': 1, 'none':1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)
    
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)
    
    ### précision
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Précision: {accuracy:.3f}')
        
    ### rapport de classification
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nRapport de classification:')
    print(class_report)
    
    ### matrice de confusion
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nMatrice de Confusion:')
    print(conf_matrix)

## Tester Llama2 avant fine-tuning 👨🏾‍💻

Ensuite, nous devons nous occuper du modèle, qui est un 7b-hf (7 milliards de paramètres, sans RLHF, au format compatible avec HuggingFace), en le chargeant à partir des modèles Kaggle et en le quantifiant.

#### Chargement et quantification du modèle :

* D'abord, le code charge le modèle de langage Llama-2 depuis le Hub Hugging Face.
* Ensuite, le code obtient le type de données float16 de la bibliothèque torch. C'est le type de données qui sera utilisé pour les calculs.
* Ensuite, il crée un objet BitsAndBytesConfig avec les paramètres suivants :
    1. load_in_4bit : charge les poids du modèle au format 4 bits.
    2. bnb_4bit_quant_type : utilise le type de quantification "nf4". Le 4-bit NormalFloat (NF4) est un nouveau type de données théoriquement optimal pour les poids distribués normalement.
    3. bnb_4bit_compute_dtype : utilise le type de données float16 pour les calculs.
    4. bnb_4bit_use_double_quant : pour la double quantification (réduit l'empreinte mémoire moyenne en quantifiant également les constantes de quantification et économise 0,4 bits supplémentaires par paramètre).
* Ensuite, le code crée un objet AutoModelForCausalLM à partir du modèle de langage pré-entraîné Llama-2, en utilisant l'objet BitsAndBytesConfig pour la quantification.
* Après cela, le code désactive la mise en cache pour le modèle.
* Enfin, le code définit la probabilité des tokens de pré-entraînement à 1.


#### Chargement du tokenizer :

* D'abord, le code charge le tokenizer pour le modèle de langage Llama-2.
* Ensuite, il définit le token de padding comme étant le token de fin de séquence (EOS).
* Enfin, le code définit le côté de padding sur "right", ce qui signifie que les séquences d'entrée seront remplies du côté droit. C'est crucial pour une direction de padding correcte (c'est ainsi que fonctionne Llama 2).

In [11]:
model_name = "togethercomputer/LLaMA-2-7B-32K"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config, 
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model, tokenizer = setup_chat_format(model, tokenizer)

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/942 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Dans la cellule suivante, nous définissons une fonction pour prédire le sentiment d'un titre d'actualité en utilisant le modèle de langage Llama-2. La fonction prend trois arguments :

- `test` : un DataFrame Pandas contenant les titres d'actualités à prédire.
- `model` : le modèle de langage pré-entraîné Llama-2.
- `tokenizer` : le tokenizer pour le modèle de langage Llama-2.

La fonction fonctionne comme suit :

1. Pour chaque titre d'actualité dans le DataFrame `test` :
    * Créer un prompt pour le modèle de langage, lui demandant d'analyser le sentiment du titre d'actualité et de renvoyer l'étiquette de sentiment correspondante.
    * Utiliser la fonction `pipeline()` de la bibliothèque Hugging Face Transformers pour générer du texte à partir du modèle de langage, en utilisant le prompt.
    * Extrayer l'étiquette de sentiment prédite du texte généré.
    * Ajouter l'étiquette de sentiment prédite à la liste `y_pred`.
2. Retournez la liste `y_pred`.

In [12]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens = 1, 
                        temperature = 1,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
    return y_pred

À ce stade, nous sommes prêts à tester le modèle Llama 2 7b-hf et à évaluer ses performances sur notre problème sans aucun ajustement fin. Cela nous permet d'obtenir des informations sur le modèle lui-même et d'établir une ligne de base.

In [13]:
y_pred = predict(test, model, tokenizer)

100%|██████████| 900/900 [05:59<00:00,  2.50it/s]


Dans la cellule suivante, nous évaluons les résultats. Il y a peu à dire, les performances sont vraiment terribles car le modèle 7b-hf a tendance à prédire principalement un sentiment neutre et détecte rarement les sentiments positifs ou négatifs.

In [14]:
evaluate(y_true, y_pred)

Précision: 0.339

Rapport de classification:
              precision    recall  f1-score   support

           0       1.00      0.00      0.01       300
           1       0.33      1.00      0.50       300
           2       0.83      0.02      0.03       300

    accuracy                           0.34       900
   macro avg       0.72      0.34      0.18       900
weighted avg       0.72      0.34      0.18       900


Matrice de Confusion:
[[  1 299   0]
 [  0 299   1]
 [  0 295   5]]


## Fine-tuning 🦾

Dans la cellule suivante, nous préparons tout pour le fine-tuning. Nous configurons et initialisons un entraîneur **Simple Fine-tuning Trainer (SFTTrainer)** pour entraîner le LLM en utilisant la méthode d'**ajustement efficace des paramètres (PEFT)**, ce qui devrait économiser du temps car elle opère sur un nombre réduit de paramètres par rapport à la taille totale du modèle.  

La méthode PEFT se concentre sur le raffinement d'un ensemble limité de paramètres supplémentaires du modèle, tout en maintenant la majorité des paramètres du LLM pré-entraîné fixes. Cela réduit considérablement les coûts à la fois computationnels et de stockage. De plus, cette stratégie adresse le défi de l'oubli catastrophique, qui se produit souvent lors du fine-tuning complet des LLMs.  

[**Détails PEFT:** Voir ressources de la Séance 7](https://github.com/ANYANTUDRE/Stage-IA-Selever-GO-AI-Corp/blob/main/02.%20Supports%20de%20Cours%20-%20Formations/07.%20S%C3%A9ance%207%20-%20M%C3%A9triques%20%26%20%20PEFTs%20-%20LoRA%20%26%20QLoRA/03.%20Parameter%20efficient%20Fine-tuning%20(PEFT).pdf)


#### PEFTConfig :

L'objet `peft_config` spécifie les paramètres pour PEFT. Voici quelques paramètres importants :

- `lora_alpha` : taux d'apprentissage pour les matrices de mise à jour LoRA.
- `lora_dropout` : probabilité de dropout pour les matrices de mise à jour LoRA (permet d'éviter l'overfitting)
- `r` : rang des matrices de mise à jour LoRA.
- `bias` : type de biais à utiliser. Les valeurs possibles sont none, additive et learned.
- `task_type` : type de tâche pour laquelle le modèle est entraîné. Les valeurs possibles sont CAUSAL_LM et MASKED_LM.

#### TrainingArguments :

L'objet `training_arguments` spécifie les paramètres pour l'entraînement du modèle. Voici quelques paramètres importants :

- `output_dir` : répertoire où les journaux d'entraînement et les checkpoints seront sauvegardés.
- `num_train_epochs` : nombre d'époques pour entraîner le modèle.
- `per_device_train_batch_size` : nombre d'échantillons dans chaque batch sur chaque dispositif (ici on utilise un seul GPU)
- `gradient_accumulation_steps` : nombre de batches pour accumuler les gradients avant de mettre à jour les paramètres du modèle.
- `optim` :optimiseur à utiliser pour l'entraînement du modèle.
- `save_steps` : nombre d'étapes après lesquelles sauvegarder un checkpoint.
- `logging_steps` : nombre d'étapes après lesquelles enregistrer les métriques d'entraînement.
- `learning_rate` : taux d'apprentissage pour l'optimiseur.
- `weight_decay` : paramètre de dégradation du poids pour l'optimiseur.
- `fp16` : utilisation ou non de la précision de virgule flottante 16 bits.
- `bf16` : utilisation ou non de la précision BFloat16.
- `max_grad_norm` : norme maximale du gradient.
- `max_steps` : nombre maximal d'étapes pour entraîner le modèle.
- `warmup_ratio` : proportion des étapes d'entraînement à utiliser pour le réchauffement du taux d'apprentissage.
- `group_by_length` : grouper ou non les échantillons d'entraînement par longueur.
- `lr_scheduler_type` : type de planificateur de taux d'apprentissage à utiliser.
- `report_to` : outils auxquels rapporter les métriques d'entraînement.
- `evaluation_strategy` : stratégie pour évaluer le modèle pendant l'entraînement.


#### SFTTrainer :

Le SFTTrainer est une classe d'entraîneur personnalisée de la bibliothèque TRL. Il est utilisé pour entraîner les LLMs (utilisant également la méthode PEFT).

L'objet SFTTrainer est initialisé avec les arguments suivants :

- `model` : modèle à entraîner.
- `train_dataset` : dataset d'entraînement.
- `eval_dataset` : dataset d'évaluation.
- `peft_config` : configuration PEFT.
- `dataset_text_field` : nom du champ texte dans le dataset.
- `tokenizer` : tokenizer à utiliser.
- `args` : arguments d'entraînement.
- `packing` : indique si les échantillons d'entraînement doivent être empaquetés.
- `max_seq_length` : longueur maximale de la séquence.

Une fois que l'objet SFTTrainer est initialisé, il peut être utilisé pour entraîner le modèle en appelant la méthode `train()`.

In [15]:
peft_config = LoraConfig(
        lora_alpha=32, 
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir="trained_weigths",             ### répertoire pour sauvegarder et identifiant du référentiel
    num_train_epochs=1,                       ### nombre d'époques d'entraînement
    per_device_train_batch_size=1,            ### taille du batch par périphérique pendant l'entraînement
    gradient_accumulation_steps=4,            ### nombre d'étapes avant d'effectuer une passe de rétropropagation/mise à jour
    gradient_checkpointing=True,              ### utilise le point de contrôle de gradient pour économiser de la mémoire
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         ### enregistre chaque 25 étapes
    learning_rate=2e-4,                       ### taux d'apprentissage, basé sur le papier QLoRA
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        ### norme maximale du gradient basée sur le papier QLoRA
    max_steps=-1,
    warmup_ratio=0.03,                        ### ratio de préchauffage basé sur le papier QLoRA
    group_by_length=True,
    lr_scheduler_type="cosine",               ### utilise le programmeur de taux d'apprentissage cosinus
    report_to="tensorboard",                  ### rapporte les métriques à tensorboard
    evaluation_strategy="epoch"               ### sauvegarde le checkpoint à chaque époque
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Le code suivant va entraîner le modèle en utilisant la méthode `trainer.train()` et ensuite sauvegarder le modèle entraîné dans le répertoire `trained-model`. En utilisant le GPU standard P100 offert par Kaggle, l'entraînement devrait être assez rapide.

In [16]:
# train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.596300,0.520684


TrainOutput(global_step=225, training_loss=0.6397684393988715, metrics={'train_runtime': 1319.221, 'train_samples_per_second': 0.682, 'train_steps_per_second': 0.171, 'total_flos': 4056091312422912.0, 'train_loss': 0.6397684393988715, 'epoch': 1.0})

Le modèle et le tokenizer sont sauvegardés sur le disque pour une utilisation ultérieure.

In [17]:
trainer.save_model()
tokenizer.save_pretrained("trained_weigths")

('trained_weigths/tokenizer_config.json',
 'trained_weigths/special_tokens_map.json',
 'trained_weigths/tokenizer.model',
 'trained_weigths/added_tokens.json',
 'trained_weigths/tokenizer.json')

Ensuite, charger l'extension TensorBoard et démarrer TensorBoard, en pointant vers le répertoire `logs/runs`, qui est censé contenir les journaux d'entraînement et les checkpoints de votre modèle, vous permettra de comprendre comment le modèle s'ajuste pendant l'entraînement.

In [18]:
%load_ext tensorboard
%tensorboard --logdir logs/runs

## Sauvegarde du modèle 💾

À ce stade, afin de démontrer comment réutiliser le modèle, nous le rechargeons à partir du disque et le fusionnons avec le modèle LLama d'origine.

En fait, lorsque nous travaillons avec QLoRA, nous n'entraînons exclusivement que des adaptateurs au lieu du modèle entier. Ainsi, lorsque vous sauvegardez le modèle pendant l'entraînement, vous ne conservez que les poids des adaptateurs, pas l'ensemble du modèle. Si vous souhaitez sauvegarder le modèle complet pour une utilisation plus facile avec l'inférence de génération de texte, vous pouvez fusionner les poids des adaptateurs avec les poids du modèle en utilisant la méthode `merge_and_unload`. Ensuite, vous pouvez sauvegarder le modèle en utilisant la méthode `save_pretrained`. Cela créera un modèle par défaut prêt pour les tâches d'inférence.

Avant de continuer, nous commençons par supprimer le modèle précédent et nettoyer la mémoire des différents objets que nous n'utiliserons plus.

In [19]:
import gc

del [model, tokenizer, peft_config, trainer, train_data, eval_data, bnb_config, training_arguments]
del [df, X_train, X_eval]
del [TrainingArguments, SFTTrainer, LoraConfig, BitsAndBytesConfig]

In [20]:
for _ in range(100):
    torch.cuda.empty_cache()
    gc.collect()

In [21]:
!nvidia-smi

Sun Jul  7 12:39:03 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0             37W /  250W |     361MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Ensuite, nous pouvons procéder à la fusion des poids et utiliser le modèle fusionné à des fins de test.

In [22]:
from peft import AutoPeftModelForCausalLM

finetuned_model = "./trained_weigths/"
compute_dtype = getattr(torch, "float16")
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoPeftModelForCausalLM.from_pretrained(
     finetuned_model,
     torch_dtype=compute_dtype,
     return_dict=False,
     low_cpu_mem_usage=True,
     device_map=device,
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("./merged_model",safe_serialization=True, max_shard_size="2GB")
tokenizer.save_pretrained("./merged_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('./merged_model/tokenizer_config.json',
 './merged_model/special_tokens_map.json',
 './merged_model/tokenizer.model',
 './merged_model/added_tokens.json',
 './merged_model/tokenizer.json')

## Test 👨🏾‍💻

Le code suivant prédira d'abord les étiquettes de sentiment pour l'ensemble de test en utilisant la fonction `predict()`. Ensuite, il évaluera les performances du modèle sur l'ensemble de test en utilisant la fonction `evaluate()`. Les résultats devraient maintenant être impressionnants avec une précision globale de plus de 0.8 et une précision élevée, ainsi qu'un rappel élevé pour les étiquettes de sentiment individuelles. La prédiction de l'étiquette neutre peut encore être améliorée, mais il est impressionnant de voir ce qui peut être réalisé avec peu de données et un peu d'ajustement fin.

In [23]:
y_pred = predict(test, merged_model, tokenizer)
evaluate(y_true, y_pred)

100%|██████████| 900/900 [04:20<00:00,  3.45it/s]

Précision: 0.856

Rapport de classification:
              precision    recall  f1-score   support

           0       0.96      0.95      0.96       300
           1       0.77      0.80      0.79       300
           2       0.83      0.81      0.82       300

    accuracy                           0.86       900
   macro avg       0.86      0.86      0.86       900
weighted avg       0.86      0.86      0.86       900


Matrice de Confusion:
[[285  15   0]
 [ 10 241  49]
 [  1  55 244]]


Le code suivant créera un DataFrame Pandas appelé `evaluation` contenant le texte, les étiquettes réelles et les étiquettes prédites de l'ensemble de test. Cela est particulièrement utile pour comprendre les erreurs que le modèle finement réglé commet et obtenir des insights sur la façon d'améliorer le prompt.

In [24]:
evaluation = pd.DataFrame({'text': X_test["text"], 
                           'y_true':y_true, 
                           'y_pred': y_pred},
                         )
evaluation.to_csv("test_predictions.csv", index=False)